In [1]:
# Load feature vectors:

import cPickle

with open('/Users/sunyambagga/Desktop/combined_feature_vector3_cbow.pkl', 'rb') as f:
    inp = cPickle.load(f)
print "Feature vectors: ", len(inp)

avg = 0
for l in inp.values():
    avg += l['competitor_num']
    
print "Average number of competitor answers: ", avg/len(inp)

Feature vectors:  411789
Average number of competitor answers:  3.62847963399


In [2]:
fv = {}

ORDER = ['url_tag', 'pic', 'code', 'ans_len', 'readability1', 'readability2', 'QA_sim', 'timeSlot', 'ave_ans_sim', 
         'min_ans_sim', 'max_ans_sim', 'competitor_num', 'ans_index', 'owner_scoreA', 'owner_scoreQ']

for ansID in inp.keys():
    temp = []
    for feature_name in ORDER:
        temp.append(inp[ansID][feature_name])
    
    fv[ansID] = temp
print len(fv.keys())

411789


In [3]:
import pandas as pd

# Load only relevant labels:
path = '/Users/sunyambagga/Desktop/labeled_answerIDs.csv'
blah = pd.read_csv(path)

answer_ids_available = inp.keys()

labels = blah.loc[blah['Answer_ID'].isin(answer_ids_available)]
print labels.nunique()
labels.head()

Answer_ID    411789
Label             2
dtype: int64


,Answer_ID,Label
3,9437192,BestAnswer
4,9437193,BestAnswer
9,19922963,BestAnswer
10,14854830,BestAnswer
14,3145756,BestAnswer


In [4]:
lab = labels.set_index('Answer_ID').T.to_dict('records')
LABELS = lab[0]
LABELS

{3670017: 'Non_BestAnswer',
 9437192: 'BestAnswer',
 9437193: 'BestAnswer',
 30408714: 'BestAnswer',
 9437202: 'Non_BestAnswer',
 19922963: 'BestAnswer',
 5417646: 'Non_BestAnswer',
 3145756: 'BestAnswer',
 26214440: 'BestAnswer',
 2097197: 'Non_BestAnswer',
 3142456: 'Non_BestAnswer',
 27263025: 'Non_BestAnswer',
 5242930: 'Non_BestAnswer',
 30408755: 'Non_BestAnswer',
 13631541: 'BestAnswer',
 31457335: 'Non_BestAnswer',
 35651642: 'BestAnswer',
 3145790: 'Non_BestAnswer',
 28311619: 'BestAnswer',
 14680133: 'BestAnswer',
 28311625: 'BestAnswer',
 30408779: 'Non_BestAnswer',
 3145804: 'Non_BestAnswer',
 36700237: 'BestAnswer',
 11534415: 'BestAnswer',
 24117328: 'BestAnswer',
 15728722: 'BestAnswer',
 3145811: 'BestAnswer',
 35651671: 'Non_BestAnswer',
 4194392: 'Non_BestAnswer',
 9437273: 'Non_BestAnswer',
 11534427: 'Non_BestAnswer',
 14680161: 'Non_BestAnswer',
 4194402: 'BestAnswer',
 27263075: 'Non_BestAnswer',
 14680165: 'Non_BestAnswer',
 35651686: 'Non_BestAnswer',
 1048685: 

In [8]:
hmm = labels.loc[labels['Label']=='BestAnswer']
hmm.nunique()

Answer_ID    139660
Label             1
dtype: int64

#### Total number of answers: 411789
#### Number of best answers: 139660
#### Number of Non-best answers: 272129

## Classes imbalanced. 3 approaches:
- As is.
- Undersampling (make them balanced by removing NonBestAnswers)
- Oversampling (using SMOTE: https://www.jair.org/media/953/live-953-2037-jair.pdf)

## As is:

In [9]:
import numpy as np

X = []
Y = []

for answer_id in fv.keys():
    X.append(fv[answer_id])
    if LABELS[answer_id] == 'BestAnswer':
        Y.append(1)
    elif LABELS[answer_id] == 'Non_BestAnswer':
        Y.append(0)
    else:
        print "WTF"

print "Total X: ", len(X)
print "Total Y: ", len(Y)

# Train test split:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

print "Train: ", len(X_train)
print len(Y_train)
print "Test: ", len(X_test)
print len(Y_test)

Total X:  411789
Total Y:  411789
Train:  329431
329431
Test:  82358
82358


In [10]:
c_tr = 0
c_test = 0
for i in Y_train:
    if i == 1:
        c_tr += 1
        
for k in Y_test:
    if k == 1:
        c_test += 1

print c_tr
print c_test
print "It's a stratified split."

111932
27728
It's a stratified split.


In [11]:
from sklearn.ensemble import RandomForestClassifier

# from sklearn.linear_model import LogisticRegression

# lr = LogisticRegression(C=1000)
# lr.fit(finalX_train, finalY_train)

# 'bootstrap': True,
#  'max_depth': 20,
#  'max_features': 'auto',
#  'min_samples_leaf': 1,
#  'min_samples_split': 2,
#  'n_estimators': 1000

rf = RandomForestClassifier(n_estimators=1000, min_samples_split=2, min_samples_leaf=1, 
                            max_features='auto', max_depth=20, bootstrap=True)
rf.fit(X_train, Y_train)

preds = rf.predict(X_test)
print len(preds)

82358


In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

print "Precision: ", precision_score(Y_test, preds)
print "Recall: ", recall_score(Y_test, preds)
print "F1 score: ", f1_score(Y_test, preds)
print "Accuracy: ", accuracy_score(Y_test, preds)
print "\nConfusion Matrix:\n", confusion_matrix(Y_test, preds)

Precision:  0.6011094626846417
Recall:  0.3360862665897288
F1 score:  0.43112581249566284
Accuracy:  0.7013890575293232

Confusion Matrix:
[[48446  6184]
 [18409  9319]]


In [13]:
# Results on 400k data (with CBOW):
# Precision:  0.5912863070539419
# Recall:  0.34946624350836697
# F1 score:  0.4392964003989482
# Accuracy:  0.6996527356176692

# Confusion Matrix:
# [[47932  6698]
#  [18038  9690]]

In [14]:
# # Results on 130k data:
# Precision:  0.6018202502844141
# Recall:  0.10238049158118831
# F1 score:  0.17499173006946742
# Accuracy:  0.8141302727679237

# Confusion Matrix:
# [[21319   350]
#  [ 4638   529]]

In [15]:
for (imp, name) in zip(rf.feature_importances_, ORDER):
    print str(name) + ": " + str(imp)

url_tag: 0.018764476039167446
pic: 0.002124803695665055
code: 0.050299504605207934
ans_len: 0.0889539982966737
readability1: 0.0700139718042633
readability2: 0.07401351972435628
QA_sim: 0.07622576772775556
timeSlot: 0.08952468975742782
ave_ans_sim: 0.07165771050274528
min_ans_sim: 0.07389680326392625
max_ans_sim: 0.07367574468647163
competitor_num: 0.06741150148007262
ans_index: 0.09513031921771803
owner_scoreA: 0.09751613205428872
owner_scoreQ: 0.05079105714426074


## Undersampling (removing non-best answers)

In [16]:
ba_train = 0
ba_test = 0

for i in Y_train:
    if i == 1:
        ba_train += 1
        
for i in Y_test:
    if i == 1:
        ba_test += 1

print "Best answers in train: ", ba_train
print "Best answers in test: ", ba_test

Best answers in train:  111932
Best answers in test:  27728


In [17]:
# Make classes balanced: ONLY for training!
# Undersampling using Provost (year 2000)

finalX_train = []
finalY_train = []

np.random.seed(7)

c_tr = 0
c_test = 0

for (x,y) in zip(X_train, Y_train):
    if y == 1:
        finalX_train.append(x)
        finalY_train.append(y)
        
    elif y == 0:
        # Stop at ba_train:
        if c_tr >= ba_train:
            continue
            
        if np.random.random() < 0.8:
            finalX_train.append(x)
            finalY_train.append(y)
            c_tr += 1

print len(finalX_train)
print len(finalY_train)

print "Total dataset: ", len(finalX_train)+len(X_test)

223864
223864
Total dataset:  306222


In [22]:
counter = 0
for i in Y_test:
    if i == 1:
        counter += 1
print "Positive Y test: ", counter

Positive Y test:  27728


In [23]:
counter = 0
for i in finalY_train:
    if i == 0:
        counter += 1
print "Positive Y train: ", counter

Positive Y train:  111932


In [24]:
# from sklearn import preprocessing
# std_scale = preprocessing.StandardScaler().fit(X_train)
# train_std = std_scale.transform(X_train)

In [26]:
from sklearn.ensemble import RandomForestClassifier
# from sklearn.svm import SVC
# sample_weight = np.array([6 if i == 1 else 1 for i in Y_train])

# svc = SVC(kernel='rbf', verbose=5.0)
# svc.fit(finalX_train, finalY_train)

# lr = LogisticRegression(C=1000)
# lr.fit(finalX_train, finalY_train)

rf = RandomForestClassifier(n_estimators=1000, min_samples_split=2, min_samples_leaf=1, 
                            max_features='auto', max_depth=20, bootstrap=True)

rf.fit(finalX_train, finalY_train)

preds = rf.predict(X_test)
print len(preds)

82358


In [27]:
# test_std = std_scale.transform(X_test)
# print len(test_std)
# print len(Y_test)

In [28]:
# Results on 130k:
# Precision:  0.6533678756476684
# Recall:  0.7321463131410877
# F1 score:  0.6905174774117002
# Accuracy:  0.6713510370226788

# Confusion Matrix:
# [[3144 2007]
#  [1384 3783]]

In [29]:
# Results on 400k (CBOW):
# Precision:  0.6551826649930633
# Recall:  0.7153418926716676
# F1 score:  0.6839419330367918
# Accuracy:  0.6667333248495646

# Confusion Matrix:
# [[16840 10439]
#  [ 7893 19835]]

In [31]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

print "Precision: ", precision_score(Y_test, preds)
print "Recall: ", recall_score(Y_test, preds)
print "F1 score: ", f1_score(Y_test, preds)
print "Accuracy: ", accuracy_score(Y_test, preds)
print "\nConfusion Matrix:\n", confusion_matrix(Y_test, preds)

 Precision:  0.4896664115163337
Recall:  0.7151976341604155
F1 score:  0.5813241092236211
Accuracy:  0.6531605915636611

Confusion Matrix:
[[33962 20668]
 [ 7897 19831]]


In [32]:
for (imp, name) in zip(rf.feature_importances_, ORDER):
    print str(name) + ": " + str(imp)

url_tag: 0.017683027602270297
pic: 0.0017072092907220015
code: 0.04814084339213073
ans_len: 0.0876373068305424
readability1: 0.0697401775870771
readability2: 0.07351476550424617
QA_sim: 0.07492544893432822
timeSlot: 0.08927342209544023
ave_ans_sim: 0.07060810190549711
min_ans_sim: 0.07318421438655838
max_ans_sim: 0.07254190626440916
competitor_num: 0.07104825417373174
ans_index: 0.10167437515970097
owner_scoreA: 0.09741535750385134
owner_scoreQ: 0.05090558936949365


## Now trying over-sampling:

In [33]:
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE().fit_sample(X_train, Y_train)
print len(X_train)
print len(X_resampled)

329431
434998


In [35]:
counter = 0
counter_before = 0

for i in Y_train:
    if i == 1:
        counter_before += 1

for i in y_resampled:
    if i == 1:
        counter += 1

print "Before, Y: ", len(Y_train)
print "Before, best answers: ", counter_before
print "\n"
print "Now, Y: ", len(y_resampled)
print "Now, best answers: ", counter

Before, Y:  329431
Before, best answers:  111932


Now, Y:  434998
Now, best answers:  217499


In [37]:
# from sklearn.model_selection import train_test_split
# # overX_train, overX_test, overY_train, overY_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# print "Train: ", len(X_train)
# print len(Y_train)
# print "Test: ", len(X_test)
# print len(Y_test)

In [38]:
from sklearn.ensemble import RandomForestClassifier
# from sklearn.svm import SVC
# sample_weight = np.array([6 if i == 1 else 1 for i in Y_train])

# svc = SVC(kernel='rbf', verbose=5.0)
# svc.fit(finalX_train, finalY_train)

# lr = LogisticRegression(C=1000)
# lr.fit(X_resampled, y_resampled)

rf = RandomForestClassifier(n_estimators=1000, min_samples_split=2, min_samples_leaf=1, 
                            max_features='auto', max_depth=20, bootstrap=True)

rf.fit(X_resampled, y_resampled)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [39]:
# Results on 130k:
# Precision:  0.49213911386374465
# Recall:  0.19992258563963616
# F1 score:  0.2843380126617121
# Accuracy:  0.806230436726785

# Confusion Matrix:
# [[20603  1066]
#  [ 4134  1033]]

In [40]:
# Testing on AS-IS test set:
print len(X_test)
print len(Y_test)

preds = rf.predict(X_test)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

print "Precision: ", precision_score(Y_test, preds)
print "Recall: ", recall_score(Y_test, preds)
print "F1 score: ", f1_score(Y_test, preds)
print "Accuracy: ", accuracy_score(Y_test, preds)
print "\nConfusion Matrix:\n", confusion_matrix(Y_test, preds)

82358
82358
Precision:  0.5742000383215176
Recall:  0.4323066935949221
F1 score:  0.4932515842317505
Accuracy:  0.7009397994123218

Confusion Matrix:
[[45741  8889]
 [15741 11987]]


In [41]:
# # Testing on under-sampled test set:

# newX_test = []
# newY_test = []

# for (x,y) in zip(X_test, Y_test):
#     if y == 1:
#         newX_test.append(x)
#         newY_test.append(y)
        
#     elif y == 0:
#         if np.random.random() < 0.24:
#             newX_test.append(x)
#             newY_test.append(y)

# print len(newX_test)
# print len(newY_test)

# counter = 0

# for i in newY_test:
#     if i == 1:
#         counter += 1
        
# print "Best answers in test: ", counter

# preds = rf.predict(newX_test)

# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# print "Precision: ", precision_score(newY_test, preds)
# print "Recall: ", recall_score(newY_test, preds)
# print "F1 score: ", f1_score(newY_test, preds)
# print "Accuracy: ", accuracy_score(newY_test, preds)
# print "\nConfusion Matrix:\n", confusion_matrix(newY_test, preds)

In [42]:
# # Testing on over-sampled test set:

# Xtest_resampled, ytest_resampled = SMOTE().fit_sample(X_test, Y_test)
# print len(X_test)
# print len(Xtest_resampled)
# preds = rf.predict(Xtest_resampled)
# print preds

# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# print "Precision: ", precision_score(ytest_resampled, preds)
# print "Recall: ", recall_score(ytest_resampled, preds)
# print "F1 score: ", f1_score(ytest_resampled, preds)
# print "Accuracy: ", accuracy_score(ytest_resampled, preds)
# print "\nConfusion Matrix:\n", confusion_matrix(ytest_resampled, preds)

In [43]:
for (imp, name) in zip(rf.feature_importances_, ORDER):
    print str(name) + ": " + str(imp)

url_tag: 0.078905853298682
pic: 0.0015104934138502417
code: 0.11540379808626906
ans_len: 0.050404564795497174
readability1: 0.036621786554815725
readability2: 0.039077048550978266
QA_sim: 0.03874931892015448
timeSlot: 0.05160040878660694
ave_ans_sim: 0.03635471756159628
min_ans_sim: 0.03926609746709215
max_ans_sim: 0.03741326394062289
competitor_num: 0.1502040141374482
ans_index: 0.2178762644668656
owner_scoreA: 0.07219543590641564
owner_scoreQ: 0.03441693411310537


## Clearly, the best choice is Undersampling:
## Doing Cross Validation for that case (in other notebook).

In [55]:
c1 = 0
c2 = 0
for i in y_resampled:
    if i == 1:
        c1 += 1

for j in ytest_resampled:
    if j == 1:
        c2 += 1
print c1 + c2

106421


In [11]:
# ROW in conf_mat is TRUE LABEL